In [1]:
import pandas as pd

In [2]:
df_retail = pd.read_csv('../data/retailer.csv')
programatic = pd.read_csv('../data/programmatic_publisher.csv')
map = pd.read_csv('../data/mapping_transac_publisher_tv.csv')
tv = pd.read_csv('../data/tv_publisher.csv')


In [3]:
unique_customer_retail = set(df_retail['customer_id'].to_list())
map = map[map['customer_id'].isin(unique_customer_retail)]

In [4]:
pro_map = map[map['dsp_id']!='unknown'].merge(programatic, on='dsp_id', how='left')
pro_map = pro_map[pro_map['customer_id']!='unknown']

In [5]:
tv_map = map[map['device_id']!='unknown'].merge(tv, on='device_id', how='left')
tv_map = tv_map[tv_map['customer_id']!='unknown']

In [6]:
df_retail = df_retail[df_retail['customer_id']!='unknown']

In [7]:
pro_map['source_data'] = 'programatic'
tv_map['source_data'] = 'tv'
df_retail['source_data'] = 'retail'

In [8]:
pro_map['timestamp_utc'] = pd.to_datetime(pro_map['timestamp_utc'])
df_retail['timestamp_utc'] = pd.to_datetime(df_retail['timestamp_utc'])
tv_map['timestamp_utc'] = pd.to_datetime(tv_map['timestamp_utc'])

In [9]:
pro_map = pro_map[pro_map['timestamp_utc']>'2024-02-29 00:00:00']
df_retail = df_retail[df_retail['timestamp_utc']>'2024-02-29 00:00:00']
tv_map = tv_map[tv_map['timestamp_utc']>'2024-02-29 00:00:00']

In [10]:
pro_map.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1811293 entries, 3 to 4264085
Data columns (total 8 columns):
 #   Column           Dtype         
---  ------           -----         
 0   customer_id      object        
 1   dsp_id           object        
 2   device_id        object        
 3   timestamp_utc    datetime64[ns]
 4   campaign_name    object        
 5   device_type      object        
 6   cost_milli_cent  float64       
 7   source_data      object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 124.4+ MB


In [11]:
pro_map = pd.get_dummies(pro_map, columns=['campaign_name'], dtype=int)
pro_map = pd.get_dummies(pro_map, columns=['device_type'], dtype=int)
pro_map.drop(columns=['device_id', 'dsp_id'], inplace=True)
pro_map.rename(columns={'cost_milli_cent':'cost_milli_cent_programmatic'}, inplace=True)

In [12]:
pro_map

,customer_id,timestamp_utc,cost_milli_cent_programmatic,source_data,campaign_name_Contextual,campaign_name_Retargeting,device_type_PC,device_type_Phone,device_type_TV,device_type_Unknown
3,reutQ3jiBX9Li4Ggqi,2024-06-16 20:55:27,601.930,programatic,1,0,0,0,1,0
4,reH7UgH29AreRh8wWy,2024-06-26 19:15:54,1191.750,programatic,1,0,0,0,1,0
5,reH7UgH29AreRh8wWy,2024-05-31 18:38:54,739.262,programatic,0,1,0,0,1,0
6,reH7UgH29AreRh8wWy,2024-06-05 22:50:03,153.500,programatic,0,1,1,0,0,0
7,reHAnmLXgIZqJT0i64,2024-05-09 21:06:22,5878.010,programatic,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4264069,reekGgat3Kzr52qMUX,2024-03-13 07:54:30,425.913,programatic,0,1,0,0,1,0
4264070,reekGgat3Kzr52qMUX,2024-03-31 18:16:15,280.017,programatic,0,1,0,0,1,0
4264071,reekGgat3Kzr52qMUX,2024-04-20 00:55:23,192.684,programatic,0,1,0,0,1,0
4264077,re41cv7tOaVBHcqCPM,2024-05-27 13:11:54,2875.684,programatic,0,1,1,0,0,0


In [13]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6146286 entries, 1 to 9866045
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   customer_id    object        
 1   timestamp_utc  datetime64[ns]
 2   event_name     object        
 3   brand          object        
 4   product_name   object        
 5   sales          float64       
 6   quantity       float64       
 7   source_data    object        
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 422.0+ MB


In [14]:
tv_map.info()

<class 'pandas.core.frame.DataFrame'>
Index: 341760 entries, 0 to 1570129
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   customer_id      341760 non-null  object        
 1   dsp_id           341760 non-null  object        
 2   device_id        341760 non-null  object        
 3   timestamp_utc    341760 non-null  datetime64[ns]
 4   cost_milli_cent  341760 non-null  float64       
 5   source_data      341760 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 18.3+ MB


In [15]:
tv_map.drop(columns=['device_id', 'dsp_id'], inplace=True)
tv_map.rename(columns={'cost_milli_cent':'cost_milli_cent_tv'}, inplace=True)

In [16]:
full_history = pd.concat([df_retail, pro_map, tv_map], ignore_index=True)

In [17]:
full_history.sort_values(by=['customer_id', 'timestamp_utc'], inplace=True)

In [18]:
len(full_history[full_history['event_name']=='Order']) + len(full_history[full_history['event_name']!='Order']) == len(full_history)

True

### Order

In [19]:
order_history = full_history[(full_history['event_name']!='Product Page View') & (full_history['event_name']!='Add to cart')]

In [20]:
order_history['cost_milli_cent_programmatic'].sum()/100000

np.float64(14074.67311117)

In [21]:
order_history

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,source_data,cost_milli_cent_programmatic,campaign_name_Contextual,campaign_name_Retargeting,device_type_PC,device_type_Phone,device_type_TV,device_type_Unknown,cost_milli_cent_tv
2257678,re000fIO9QXTWYjOfn,2024-03-13 02:58:00,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,retail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2257679,re000fIO9QXTWYjOfn,2024-05-24 18:10:37,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,retail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2257680,re000fIO9QXTWYjOfn,2024-06-04 18:27:57,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,retail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2257681,re000fIO9QXTWYjOfn,2024-06-21 22:04:39,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,retail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3352445,re000pHbVOysCXRHgt,2024-03-27 16:13:49,Order,Science Diet,SD Ca Adt Lt Ckn 30lb bg,78.99,1.0,retail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203902,rezzz8WfdxS4BBGCo8,2024-04-20 21:15:13,Order,Prescription Diet,PD Ca c/d Mul Ckn&VgStew 24x5.5oz cs,60.99,1.0,retail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8062821,rezzzZvkIaiWNQ1AmV,2024-06-26 23:50:31,NaN,NaN,NaN,NaN,NaN,tv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2325.51
8062822,rezzzZvkIaiWNQ1AmV,2024-06-30 04:39:58,NaN,NaN,NaN,NaN,NaN,tv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2325.51
4142213,rezzzipns16pTCb4OS,2024-04-18 01:35:45,Order,Science Diet,SD Ca Adt Lt SB Ckn 5lb bg,19.99,1.0,retail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
order_history = order_history.sort_values(by=['customer_id', 'timestamp_utc'])

# Columns to aggregate
ad_columns = [
    'cost_milli_cent_programmatic', 'campaign_name_Contextual', 'campaign_name_Retargeting',
    'device_type_PC', 'device_type_Phone', 'device_type_TV', 'device_type_Unknown', 'cost_milli_cent_tv'
]

# Identify ad type
ad_type_column = 'source_data'  # Used to determine if it's programmatic or TV
order_indicator = 'retail'  # Orders are identified with 'retail'

# Initialize tracking
order_data = []
current_sums = {col: 0 for col in ad_columns}  # Track sums before order
count_tv = 0
count_programmatic = 0
ad_timestamps = []  # Store timestamps of all ads before order
tv_timestamps = []  # Store timestamps of TV ads before order
programmatic_timestamps = []  # Store timestamps of Programmatic ads before order

# Iterate through rows
for _, row in order_history.iterrows():
    if row['source_data'] == order_indicator:  # If it's an order
        # Compute ad frequency
        ad_frequency_count = len(ad_timestamps)

        if len(ad_timestamps) > 1:
            ad_timestamps = sorted(ad_timestamps)
            time_diffs = [(ad_timestamps[i] - ad_timestamps[i - 1]).total_seconds() / 86400 for i in range(1, len(ad_timestamps))]
            avg_time_between_ads = sum(time_diffs) / len(time_diffs)
        else:
            avg_time_between_ads = None  # Not enough data for average

        if len(tv_timestamps) > 1:
            tv_timestamps = sorted(tv_timestamps)
            tv_time_diffs = [(tv_timestamps[i] - tv_timestamps[i - 1]).total_seconds() / 86400 for i in range(1, len(tv_timestamps))]
            avg_time_between_tv_ads = sum(tv_time_diffs) / len(tv_time_diffs)
        else:
            avg_time_between_tv_ads = None  # Not enough data

        if len(programmatic_timestamps) > 1:
            programmatic_timestamps = sorted(programmatic_timestamps)
            programmatic_time_diffs = [(programmatic_timestamps[i] - programmatic_timestamps[i - 1]).total_seconds() / 86400 for i in range(1, len(programmatic_timestamps))]
            avg_time_between_programmatic_ads = sum(programmatic_time_diffs) / len(programmatic_time_diffs)
        else:
            avg_time_between_programmatic_ads = None  # Not enough data

        # Store aggregated values
        order_data.append({
            'customer_id': row['customer_id'],
            'timestamp_utc': row['timestamp_utc'],
            'event_name' : row['event_name'],
            'brand' : row['brand'],
            'product_name' : row['product_name'],
            'sales' : row['sales'],
            'quantity': row['quantity'],
            **current_sums,  # Add all aggregated columns
            'count_tv_ads': count_tv,
            'count_programmatic_ads': count_programmatic,
            'ad_frequency_count': ad_frequency_count,
            'avg_time_between_ads_days': avg_time_between_ads,
            'avg_time_between_tv_ads_days': avg_time_between_tv_ads,
            'avg_time_between_programmatic_ads_days': avg_time_between_programmatic_ads
        })

        # Reset tracking after storing order data
        current_sums = {col: 0 for col in ad_columns}
        count_tv = 0
        count_programmatic = 0
        ad_timestamps = []
        tv_timestamps = []
        programmatic_timestamps = []

    else:  # If it's an ad exposure
        for col in ad_columns:
            if pd.notna(row[col]):  # Ensure it's not NaN
                current_sums[col] += row[col]

        # Count ad types
        if row[ad_type_column] == 'programatic':
            count_programmatic += 1
            programmatic_timestamps.append(row['timestamp_utc'])
        elif row[ad_type_column] == 'tv':
            count_tv += 1
            tv_timestamps.append(row['timestamp_utc'])

        # Store timestamp for frequency calculation
        ad_timestamps.append(row['timestamp_utc'])

# Create DataFrame with orders and their pre-order ad exposure
df_orders = pd.DataFrame(order_data)


In [23]:
df_orders

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,cost_milli_cent_programmatic,campaign_name_Contextual,campaign_name_Retargeting,...,device_type_Phone,device_type_TV,device_type_Unknown,cost_milli_cent_tv,count_tv_ads,count_programmatic_ads,ad_frequency_count,avg_time_between_ads_days,avg_time_between_tv_ads_days,avg_time_between_programmatic_ads_days
0,re000fIO9QXTWYjOfn,2024-03-13 02:58:00,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
1,re000fIO9QXTWYjOfn,2024-05-24 18:10:37,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
2,re000fIO9QXTWYjOfn,2024-06-04 18:27:57,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
3,re000fIO9QXTWYjOfn,2024-06-21 22:04:39,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
4,re000pHbVOysCXRHgt,2024-03-27 16:13:49,Order,Science Diet,SD Ca Adt Lt Ckn 30lb bg,78.99,1.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934783,rezzymDnsupIX7Q3TS,2024-05-22 14:36:42,Order,Science Diet,SD Ca A7+ SB Ckn 5lb bg,19.99,1.0,1901.966,3.0,1.0,...,2.0,1.0,0.0,2325.51,1,4,5,26.884560,NaN,33.012681
934784,rezzz8WfdxS4BBGCo8,2024-03-23 18:24:01,Order,Prescription Diet,PD Ca c/d Mul Ckn&VgStew 24x5.5oz cs,60.99,1.0,146.610,1.0,0.0,...,1.0,0.0,0.0,0.00,0,1,1,NaN,NaN,NaN
934785,rezzz8WfdxS4BBGCo8,2024-04-20 21:15:13,Order,Prescription Diet,PD Ca c/d Mul Ckn&VgStew 24x5.5oz cs,60.99,1.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
934786,rezzzipns16pTCb4OS,2024-04-18 01:35:45,Order,Science Diet,SD Ca Adt Lt SB Ckn 5lb bg,19.99,1.0,0.000,0.0,0.0,...,0.0,0.0,0.0,4651.02,2,0,2,3.201007,3.201007,NaN


In [24]:
df_orders['cost_milli_cent_programmatic'].sum()/100000

np.float64(14074.673111170003)

In [30]:
df_orders.to_csv('../data/orders_ads_history.csv')

## Product page view

In [25]:
product_page_history = full_history[(full_history['event_name']!='Order') & (full_history['event_name']!='Add to cart')]

In [26]:
product_page_history['cost_milli_cent_programmatic'].sum()/100000

np.float64(14074.673111169972)

In [ ]:
product_page_history = product_page_history.sort_values(by=['customer_id', 'timestamp_utc'])

# Columns to aggregate
ad_columns = [
    'cost_milli_cent_programmatic', 'campaign_name_Contextual', 'campaign_name_Retargeting',
    'device_type_PC', 'device_type_Phone', 'device_type_TV', 'device_type_Unknown', 'cost_milli_cent_tv'
]

# Identify ad type
ad_type_column = 'source_data'  # Used to determine if it's programmatic or TV
order_indicator = 'retail'  # Orders are identified with 'retail'

# Initialize tracking
order_data = []
current_sums = {col: 0 for col in ad_columns}  # Track sums before order
count_tv = 0
count_programmatic = 0
ad_timestamps = []  # Store timestamps of all ads before order
tv_timestamps = []  # Store timestamps of TV ads before order
programmatic_timestamps = []  # Store timestamps of Programmatic ads before order

# Iterate through rows
for _, row in product_page_history.iterrows():
    if row['source_data'] == order_indicator:  # If it's an order
        # Compute ad frequency
        ad_frequency_count = len(ad_timestamps)

        if len(ad_timestamps) > 1:
            ad_timestamps = sorted(ad_timestamps)
            time_diffs = [(ad_timestamps[i] - ad_timestamps[i - 1]).total_seconds() / 86400 for i in range(1, len(ad_timestamps))]
            avg_time_between_ads = sum(time_diffs) / len(time_diffs)
        else:
            avg_time_between_ads = None  # Not enough data for average

        if len(tv_timestamps) > 1:
            tv_timestamps = sorted(tv_timestamps)
            tv_time_diffs = [(tv_timestamps[i] - tv_timestamps[i - 1]).total_seconds() / 86400 for i in range(1, len(tv_timestamps))]
            avg_time_between_tv_ads = sum(tv_time_diffs) / len(tv_time_diffs)
        else:
            avg_time_between_tv_ads = None  # Not enough data

        if len(programmatic_timestamps) > 1:
            programmatic_timestamps = sorted(programmatic_timestamps)
            programmatic_time_diffs = [(programmatic_timestamps[i] - programmatic_timestamps[i - 1]).total_seconds() / 86400 for i in range(1, len(programmatic_timestamps))]
            avg_time_between_programmatic_ads = sum(programmatic_time_diffs) / len(programmatic_time_diffs)
        else:
            avg_time_between_programmatic_ads = None  # Not enough data

        # Store aggregated values
        order_data.append({
            'customer_id': row['customer_id'],
            'timestamp_utc': row['timestamp_utc'],
            'event_name' : row['event_name'],
            'brand' : row['brand'],
            'product_name' : row['product_name'],
            'sales' : row['sales'],
            'quantity': row['quantity'],
            **current_sums,  # Add all aggregated columns
            'count_tv_ads': count_tv,
            'count_programmatic_ads': count_programmatic,
            'ad_frequency_count': ad_frequency_count,
            'avg_time_between_ads_days': avg_time_between_ads,
            'avg_time_between_tv_ads_days': avg_time_between_tv_ads,
            'avg_time_between_programmatic_ads_days': avg_time_between_programmatic_ads
        })

        # Reset tracking after storing order data
        current_sums = {col: 0 for col in ad_columns}
        count_tv = 0
        count_programmatic = 0
        ad_timestamps = []
        tv_timestamps = []
        programmatic_timestamps = []

    else:  # If it's an ad exposure
        for col in ad_columns:
            if pd.notna(row[col]):  # Ensure it's not NaN
                current_sums[col] += row[col]

        # Count ad types
        if row[ad_type_column] == 'programatic':
            count_programmatic += 1
            programmatic_timestamps.append(row['timestamp_utc'])
        elif row[ad_type_column] == 'tv':
            count_tv += 1
            tv_timestamps.append(row['timestamp_utc'])

        # Store timestamp for frequency calculation
        ad_timestamps.append(row['timestamp_utc'])

# Create DataFrame with orders and their pre-order ad exposure
df_product_page = pd.DataFrame(order_data)


In [31]:
df_product_page.to_csv('../data/product_page_ads_history.csv')

In [29]:
df_product_page

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,cost_milli_cent_programmatic,campaign_name_Contextual,campaign_name_Retargeting,...,device_type_Phone,device_type_TV,device_type_Unknown,cost_milli_cent_tv,count_tv_ads,count_programmatic_ads,ad_frequency_count,avg_time_between_ads_days,avg_time_between_tv_ads_days,avg_time_between_programmatic_ads_days
0,re000kbtVVzPwZcEr4,2024-03-06 23:57:04,Product Page View,Science Diet,SD Pup SmPws Ckn 4.5lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
1,re000kbtVVzPwZcEr4,2024-04-15 15:04:05,Product Page View,Science Diet,SD Ca Adt SenSt&Sk Ckn 30lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
2,re000kbtVVzPwZcEr4,2024-04-15 16:15:59,Product Page View,Science Diet,SD Ca Adt SenSt&Sk Ckn 30lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
3,re000kbtVVzPwZcEr4,2024-04-15 16:33:34,Product Page View,Science Diet,SD Ca Adt SenSt&Sk Ckn 30lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
4,re000kbtVVzPwZcEr4,2024-04-15 19:05:05,Product Page View,Science Diet,SD Ca Adt SenSt&Sk Ckn 30lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3908049,rezzz8a320jhOvmL3A,2024-04-26 23:11:28,Product Page View,Science Diet,SD Ca Adt LB LM&BR 33lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
3908050,rezzz8a320jhOvmL3A,2024-04-26 23:40:20,Product Page View,Science Diet,SD Ca A6+ LB Ckn 15lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
3908051,rezzzYRiwreLF23ot3,2024-06-04 23:56:40,Product Page View,Science Diet,SD Ca Adt PerWgt Ckn SB 4lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
3908052,rezzzZvkIaiWNQ1AmV,2024-04-16 22:01:01,Product Page View,Prescription Diet,PD m/d Feline 8.5lb bg,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0,0,0,NaN,NaN,NaN
